In [1]:
#downloading and appending daily stock price by minutes for usnadian and us market
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
pd.options.mode.chained_assignment = None

path = '/Users/jayren/Desktop/stock/Stock_Daily'
dailydata = '/Users/jayren/Desktop/stock/stock_Daily/dailydata'
consolidated = '/stockPrice_consolidated'

startdate=date.today()-timedelta(days=5)
dayday=date.today()-timedelta(days=0)

yy=f'{startdate:%Y}'
mm=f'{startdate:%m}'
dd=f'{startdate:%d}'

daystring=yy+mm+dd

# list of tickers

us_sp500_list = pd.read_excel(path + '/list.xlsx', sheet_name='SP500_LIST')
ca_sptsx_list = pd.read_excel(path + '/list.xlsx', sheet_name='SPTSX_LIST')

us_sp500_tickers = us_sp500_list['Symbol'].tolist()
ca_sp_tickers = ca_sptsx_list['Symbol'].tolist()

us_stock_price = pd.DataFrame()
ca_stock_price=pd.DataFrame()

# download us stock price

us_stock = yf.download(us_sp500_tickers, group_by=us_sp500_tickers, interval='1m', start=startdate, end=None)
us_stock.index.name='Datetime'

#us_stock_price.index = pd.to_datetime(us_stock_price.index, format='%Y-%m-%d')

for ticker in us_sp500_tickers:
    stock = us_stock.loc[:, ticker.upper()]
    stock.loc[:,'ticker'] = ticker.upper()
    us_stock_price = pd.concat([us_stock_price, stock])
            
us_stock_price.reset_index(inplace=True)
us_stock_price['Datetime']=us_stock_price['Datetime'].dt.tz_localize(None)
us_stock_price=us_stock_price[(us_stock_price['Volume'].notnull()) & (us_stock_price['Volume']!=0)]
us_stock_price['date']=us_stock_price['Datetime'].dt.date


# download canadian stock price
try:    
    ca_stock = yf.download(ca_sp_tickers, group_by=ca_sp_tickers, interval='1m', start=startdate, end=None)
    ca_stock.index.name='Datetime'

    for ticker in ca_sp_tickers:
        stock = ca_stock.loc[:,ticker.upper()]
        stock.loc[:,'ticker'] = ticker.upper()
        ca_stock_price = pd.concat([ca_stock_price, stock])
    
    ca_stock_price.reset_index(inplace=True)
    ca_stock_price['Datetime']=ca_stock_price['Datetime'].dt.tz_localize(None)
    ca_stock_price=ca_stock_price[(ca_stock_price['Volume'].notnull()) & (ca_stock_price['Volume']!=0)]
    ca_stock_price['date']=ca_stock_price['Datetime'].dt.date

except:
    print('No data available for today')
    
# download adhoc stock price



with pd.ExcelWriter(dailydata + '/dailyprice'+daystring+'.xlsx') as writer:
    us_stock_price.to_excel(writer, sheet_name='us_stock_price', index=False)
    ca_stock_price.to_excel(writer, sheet_name='ca_stock_price', index=False)
    # adhoc_stock_price.to_excel(writer, sheet_name='adhoc_stock_price', index=False)
    

#derive the list of date for each stock price dateset
us_date=[x.strftime('%Y-%m-%d') for x in us_stock_price['date'].unique().tolist()]
ca_date=[x.strftime('%Y-%m-%d') for x in ca_stock_price['date'].unique().tolist()]
# adhoc_date=[x.strftime('%Y-%m-%d') for x in adhoc_stock_price['date'].unique().tolist()]

#read the historical data file
us=pd.read_csv(dailydata+consolidated+'/us_stock_price.csv')
ca=pd.read_csv(dailydata+consolidated+'/ca_stock_price.csv')
# adhoc=pd.read_csv(dailydata+consolidated+'/adhoc_stock_price.csv')

#set up criterion to exclude the current dataset from existing dataset
us_criterion=lambda df: df['date'] not in us_date
ca_criterion=lambda df: df['date'] not in ca_date
# adhoc_criterion=lambda df:df['date'] not in adhoc_date

#consolidate new downloads and the historical dataset
#The dataset below are consolidated stock price dataset
us=us[us.apply(us_criterion, axis=1)]
us = pd.concat([us, us_stock_price])

ca=ca[ca.apply(ca_criterion, axis=1)]
ca=pd.concat([ca,ca_stock_price])

# adhoc=adhoc[adhoc.apply(adhoc_criterion, axis=1)]
# adhoc=pd.concat([adhoc, adhoc_stock_price])

#save to csv file
us.to_csv(dailydata+consolidated+'/us_stock_price.csv', index=False)
ca.to_csv(dailydata+consolidated+'/ca_stock_price.csv', index=False)
# adhoc.to_csv(dailydata+consolidated+'/adhoc_stock_price.csv', index=False)



/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_30584/1461917103.py:33: FutureWarning: YF.download() has changed argument auto_adjust default to True
  us_stock = yf.download(us_sp500_tickers, group_by=us_sp500_tickers, interval='1m', start=startdate, end=None)
[*********************100%***********************]  462 of 462 completed

1 Failed download:
['WBA']: YFPricesMissingError('possibly delisted; no price data found  (1m 2025-09-09 -> 2025-09-14 12:25:38-04:00)')
/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_30584/1461917103.py:51: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ca_stock = yf.download(ca_sp_tickers, group_by=ca_sp_tickers, interval='1m', start=startdate, end=None)
[*********************100%***********************]  197 of 197 completed

2 Failed downloads:
['CIX.TO']: YFPricesMissingError('possibly delisted; no price data found  (1m 2025-09-09 -> 2025-09-14 12:25:52-04:00)')
['MAG.TO']: YFPricesMissingError('

In [2]:
us.tail()


,Datetime,Open,High,Low,Close,Adj Close,Volume,ticker,date
720715,2025-09-12 19:55:00,148.710007,148.809998,148.570007,148.809998,NaN,45750.0,ZTS,2025-09-12
720716,2025-09-12 19:56:00,148.800003,148.839996,148.389999,148.389999,NaN,38165.0,ZTS,2025-09-12
720717,2025-09-12 19:57:00,148.389999,148.419998,148.259995,148.410004,NaN,46308.0,ZTS,2025-09-12
720718,2025-09-12 19:58:00,148.419998,148.460007,148.320007,148.335007,NaN,73138.0,ZTS,2025-09-12
720719,2025-09-12 19:59:00,148.335007,148.419998,148.179993,148.190002,NaN,112929.0,ZTS,2025-09-12
